# COMP90086 Week 12: Object Detection
(To be run on Colab Notebook)

In this workshop, we will discuss object detection. We will use the [Tensorflow 2 Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) with pre-trained models from the [Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) to detect objects in images. We will then visualise and evaluate the results with the open-source tool [FiftyOne](https://voxel51.com/docs/fiftyone/).


Table of Contents

- 1 Installation
    
- 2 Object Detection From Pre-trained Model
    - (1) Download the model
    - (2) Load the model
    - (3) Download the test dataset
    - (4) Make predictions
    
- 3 Evaluation
    - (1) Definitions of different metrics
    - (2) Running evaluation
    - (3) Aggregate results
    - (4) Evaluating model mAP
    - (5) Sample-level analysis
    
- 4 Exercise


> **Important**: If you're running on a local machine, be sure to refer to the file `Workshop12(Local_version)` and follow the [installation instructions](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html). This notebook includes only what's necessary to run in Colab.


This tutorial is adapted from the TensorFlow 2 Object Detection API tutorial and the FiftyOne Docs:
<table class="tfo-notebook-buttons" align="left">
    <td>
    <a target="_blank" href="http://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/?badge=latest">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png"/>
    View on Object Detection API Document</a>
  </td>   
  <td>
    <a target="_blank" href="https://voxel51.com/docs/fiftyone/">   
    <img src="https://voxel51.com/images/logo/voxel51-logo-horz-color-600dpi.png" width=117/>
    View on FiftyOne Docs</a>
  </td>
</table>

## Assignment Feedback & Tips
Overall, everyone seems to do well. Once you get it working properly, you should get most of the scores already:

### Assignment 3: Common reason of losing marks:
- 'I choose threshold of 2 pixels because the spec said so'
- 'Image didn't performed well because SIFT didn't perform well' - Why so?
- Code commenting: We need to read your code to understand what you did. If you don't guide us, we have to spend more time doing exploring, and you lose marks.
- 'I give up midway' - Code error can be forgiving. We aim to see if you understand the methods. If you can still write code to implement it, we would still give you most of the points (at the worst, you just can't review Task 3 properly, which takes only a small portion).

### Project report tips:
- It's an art to balance between detailed and conciseness: Include as much details of yours as possible, but remember to keep it short but clear!
- We always look for reasoning: If I mark your report and I have to "hmmmm? Why so?", then you lose marks.
- Visualisation is a good addition to presentation and reasoning.
- There is no right, only good arguments (definitely not the Karen style ones).
- Remember to make it readable:
  - If I don't understand, I can assume you did not do it.
  - Best to have a peer reading your report, and see if they understand what you did.


## 1 Installation

This may take up about 2 minutes.

In [ ]:
!pip install tf_slim
!pip install pycocotools
!pip install fiftyone
#!pip install googledrivedownloader

import os
import time
import json
import pathlib
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")     # Suppress warnings
import tensorflow as tf
tf.get_logger().setLevel('ERROR')     # Suppress TensorFlow logging
import fiftyone as fo
import fiftyone.zoo as foz
from PIL import Image
from google_drive_downloader import GoogleDriveDownloader as gdd

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

os.chdir('models/research/')

Enable GPU dynamic memory allocation

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## 2 Object Detection From Pre-trained Model

This demo will take you through the steps of running an "out-of-the-box" TensorFlow 2 compatible
detection model on a collection of images. We will utilise the [TensorFlow 2 Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). The TensorFlow Object Detection API is an open source framework built on top of TensorFlow that makes it easy to construct, train and deploy object detection models. 

### (1) Download the model

We will use the pre-trained models from the Model Zoo to detect objects in images. 

The [Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) is a collection of detection models pre-trained on the COCO 2017 dataset. The [COCO 2017 dataset](https://cocodataset.org/#home) is a large-scale dataset of labelled images. It contains 80 common object categories. The models can perform object detection on these object categories. You can explore the COCO dataset by using the the [COCO dataset explorer page](https://cocodataset.org/#explore).

You might find that there are plenty of models available in the Model Zoo. The architecture of all these models can be identified by name. In this workshop, we suggest you to try some of the following models. You could compare and contrast the results of the Faster R-CNN, SSD, and CenterNet. You could also try swapping the "backbone" of a network (e.g., ResNet vs. MobileNet) to see if it changes the results.

| Name | MODEL_NAME   | MODEL_DATE | Speed (ms) | COCO mAP |
|------|--------------|------------|------------|----------|
|[CenterNet Resnet50 V2 512x512](http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v2_512x512_coco17_tpu-8.tar.gz)|`centernet_resnet50_v2_512x512_coco17_tpu-8`|`20200711`|27|29.5|
|[CenterNet Resnet50 V1 FPN 512x512](http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz)|`centernet_resnet50_v1_fpn_512x512_coco17_tpu-8`|`20200711`|27|31.2|
|[CenterNet HourGlass104 512x512](http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz)|`centernet_hg104_512x512_coco17_tpu-8`|`20200713`|70|41.9|
|[SSD ResNet50 V1 FPN 640x640 (RetinaNet50)](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz)|`ssd_resnet50_v1_fpn_640x640_coco17_tpu-8`|`20200711`|46|34.3|
|[SSD ResNet101 V1 FPN 640x640 (RetinaNet101)](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz)|`ssd_resnet101_v1_fpn_640x640_coco17_tpu-8`|`20200711`|57|35.6|
|[SSD MobileNet V1 FPN 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz)|`ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8`|`20200711`|48|29.1|
|[Faster R-CNN ResNet50 V1 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz)|`faster_rcnn_resnet50_v1_640x640_coco17_tpu-8`|`20200711`|53|29.3|
|[Faster R-CNN ResNet50 V1 800x1333](http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8.tar.gz)|`faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8`|`20200711`|65|31.6|
|[Faster R-CNN ResNet101 V1 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz)  | `faster_rcnn_resnet101_v1_640x640_coco17_tpu-8` |  `20200711` | 55|   31.8 |
|[Faster R-CNN Inception ResNet V2 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz)  | `faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8` |  `20200711` | 206|   37.7 |

More models can be found in the [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).

#### Metrics in the table
    - Speed (ms) is the expected response time to get a response when querying the model, the lower the better.  
    - COCO mAP is the mean average precision score against the COCO dataset of images, the higher the better.

The code snippet shown below is used to download the pre-trained object detection model we shall
use to perform inference. The particular pre-trained model we will use is the `CenterNet MobileNetV2 FPN 512x512`.  To use a different model you will need to:

1. Copy the `MODEL_NAME` part of the table and use it to replace the value of the `MODEL_NAME` variable in the code shown below;
2. Copy the `MODEL_DATE` part of the table and use it to replace the value of the `MODEL_DATE` variable in the code shown below.

For example, if we want to switch to the `CenterNet Resnet50 V1 FPN 512x512` model, then the following block should use code:
```
MODEL_DATE = '20200711'
MODEL_NAME = 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8'
```

More information on the functions used below, see:
- [`tf.keras.utils.get_file( )`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file) Downloads a file from a URL if it not already in the cache.

In [ ]:
# Download and extract model
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + model_file,
                                        untar=True, #Whether the file should be decompressed
                                        cache_dir="../../") #Location to store cached files
    return str(model_dir)
MODEL_DATE = '20200711'
MODEL_NAME = 'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)

90464256/90453990 [==============================] - 1s 0us/step


### (2) Load the model
Next we load the downloaded model. In this example we will be using
the [Saved Model Format](https://www.tensorflow.org/guide/saved_model) to load the model.

In [ ]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 18.902263879776 seconds


### (3) Download the test dataset

To speed up the experiment, the test set we used in the workshop contained only fifty images. These images and their ground truths are taken from the [COCO 2017 Validation dataset](https://cocodataset.org/#download) (Note: The official COCO test set annotations are unavailable to the public).

In [ ]:
gdd.download_file_from_google_drive(file_id='1PDUnoCwm2LrYRkDbhjmUEJaPK0EH-LNU',
                                    dest_path='../../datasets.zip',
                                    unzip=True)

Unzipping...Done.


In [ ]:
# Load COCO formatted dataset ground_truth
coco_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path='../../datasets/images/',
    labels_path='../../datasets/coco.json',
    include_id=True,
    label_field="ground_truth",
)

 100% |███████████████████| 50/50 [383.4ms elapsed, 0s remaining, 132.5 samples/s]      


Let’s inspect the dataset

In [ ]:
# Print some information about the dataset
print(coco_dataset)

Name:        2021.10.16.06.56.52
Media type:  image
Num samples: 50
Persistent:  False
Tags:        []
Sample fields:
    id:                      fiftyone.core.fields.ObjectIdField
    filepath:                fiftyone.core.fields.StringField
    tags:                    fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth_detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    ground_truth_coco_id:    fiftyone.core.fields.IntField


In [ ]:
# Print a ground truth detection
sample = coco_dataset.first()
print(sample.ground_truth_detections.detections[0])

<Detection: {
    'id': '616a77b536fd785cd314ebf2',
    'attributes': BaseDict({}),
    'tags': BaseList([]),
    'label': 'person',
    'bounding_box': BaseList([
        0.30445833333333333,
        0.46070312500000005,
        0.22441666666666665,
        0.293421875,
    ]),
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'person',
    'iscrowd': 0,
}>


In [ ]:
# Export labels in COCO format
with open("../../datasets/coco.json") as f:
    data = json.load(f)

# Explore labels    
data['categories']

[{'id': 1, 'name': 'person', 'supercategory': 'person'},
 {'id': 2, 'name': 'bicycle', 'supercategory': 'vehicle'},
 {'id': 3, 'name': 'car', 'supercategory': 'vehicle'},
 {'id': 4, 'name': 'motorcycle', 'supercategory': 'vehicle'},
 {'id': 5, 'name': 'airplane', 'supercategory': 'vehicle'},
 {'id': 6, 'name': 'bus', 'supercategory': 'vehicle'},
 {'id': 7, 'name': 'train', 'supercategory': 'vehicle'},
 {'id': 8, 'name': 'truck', 'supercategory': 'vehicle'},
 {'id': 9, 'name': 'boat', 'supercategory': 'vehicle'},
 {'id': 10, 'name': 'traffic light', 'supercategory': 'outdoor'},
 {'id': 11, 'name': 'fire hydrant', 'supercategory': 'outdoor'},
 {'id': 13, 'name': 'stop sign', 'supercategory': 'outdoor'},
 {'id': 14, 'name': 'parking meter', 'supercategory': 'outdoor'},
 {'id': 15, 'name': 'bench', 'supercategory': 'outdoor'},
 {'id': 16, 'name': 'bird', 'supercategory': 'animal'},
 {'id': 17, 'name': 'cat', 'supercategory': 'animal'},
 {'id': 18, 'name': 'dog', 'supercategory': 'animal'},

#### Visualizing and exploring your dataset with FiftyOne App

[FiftyOne](https://voxel51.com/docs/fiftyone/index.html#) is a open-source tool for building datasets and computer vision models. It enables you to visualize datasets and interpret models faster and more effectively.

In [ ]:
session = fo.launch_app(coco_dataset)

#### (Optional) Split your own sub-dataset from the COCO

You could also split your own sub-dataset from the COCO 2017 validation dataset.

Below we will load 50 random samples from the COCO 2017 validation split that contain cats and dogs. Images that contain all `classes` will be prioritized first, followed by images that contain at least one of the required `classes`. If there are not enough images matching `classes` in the split to meet `max_samples`, only the available images will be loaded. By default, only detections are loaded. By setting `shuffle=True`, it will randomly shuffle the order in which samples are chosen for partial downloads.

> **Important**: If you do not intend to split your own dataset, **do not execute the one block of code below**. Once the following one block is executed, FiftyOne will perform a download, which can be very large.

In [ ]:
# import fiftyone.zoo as foz

# my_dataset = foz.load_zoo_dataset(
#     "coco-2017",
#     split="validation",
#     classes=["cat", "dog"],
#     max_samples=50,
#     shuffle=True,
# )

# # Print some information about the dataset
# print(my_dataset)

> If your are using this `my_dataset`: please replace `coco_dataset` with `my_dataset`, and replace `ground_truth_detections` with `ground_truth` in all the following code.

In addition to loading the COCO datasets themselves,  you could load your own datasets stored in [COCO format](https://cocodataset.org/#format-data) using [this guide](https://voxel51.com/docs/fiftyone/integrations/coco.html#loading-coco-formatted-data).

### (4) Make predictions

Install Tensorflow 2 Object Detection API

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=2020-resolver .

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

Now, we will use the chosen pre-trained model to make predictions on our dataset.

You could set `min_score_thresh` to other values (between 0 and 1) to allow more detections in or to filter out more detections.

In [ ]:
min_score_thresh=.30

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

# Choose a random subset of 25 samples to add predictions.
# The larger the sample you choose, the longer it will take.
predictions_view = coco_dataset.take(25, seed=51)

# Get class list
classes = coco_dataset.default_classes

# Add predictions to samples
with fo.ProgressBar() as pb:
    for sample in pb(predictions_view):
        # Load image
        image_np = load_image_into_numpy_array(sample.filepath)

        # Things to try:
        # Convert image to grayscale
        # Here we still expect the input image to have 3 channels
        
        # image_np = np.tile(
        #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)
        
        
        
        # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
        input_tensor = tf.convert_to_tensor(image_np)
        
        # The model expects a batch of images, so add an axis with `tf.newaxis`.
        input_tensor = input_tensor[tf.newaxis, ...]

        # input_tensor = np.expand_dims(image_np, 0)
        detections = detect_fn(input_tensor)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                       for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        image_np_with_detections = image_np.copy()

        result_detections=[]
        for label, score, box in zip(detections['detection_classes'], detections['detection_scores'], detections['detection_boxes']):
            # Convert to [top-left-x, top-left-y, width, height]            
            ymin, xmin, ymax, xmax = box
            rel_box = [xmin, ymin, (xmax - xmin), (ymax - ymin)]

            # result_detections.append({"image_id": 1, "category_id": label, "bbox": rel_box, "score": score})
            if score>min_score_thresh:
                result_detections.append(
                    fo.Detection(
                        label=classes[label],
                        bounding_box=rel_box,
                        confidence=score
                    )
                )
                
        # Save predictions to dataset
        sample["mytest"] = fo.Detections(detections=result_detections)
        sample.save()

 100% |███████████████████| 25/25 [25.9s elapsed, 0s remaining, 3.4 samples/s]      


As our data is stored in COCO format, it can be loaded into FiftyOne and then visualised in this application. Let’s load `predictions_view` in the FiftyOne to visualize the predictions that we added:

In [ ]:
session.view = predictions_view

## 3 Evaluation

Now that we have samples with ground truth and predicted objects, let’s use FiftyOne to evaluate the quality of the detections. FiftyOne provides a powerful [evaluation API](https://voxel51.com/docs/fiftyone/user_guide/evaluation.html) that contains a collection of methods for performing evaluation of model predictions. Since we’re working with object detections here, we’ll use [detection evaluation](https://voxel51.com/docs/fiftyone/user_guide/evaluation.html#detections).

### (1) Definitions of different metrics

Adapted from [here](https://github.com/rafaelpadilla/Object-Detection-Metrics#important-definitions) and [here](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173).

#### True Positive, False Positive, False Negative and True Negative
Some basic concepts used by the metrics:
    
- True Positive (TP): A correct detection. Detection with IOU ≥ threshold
- False Positive (FP): A wrong detection. Detection with IOU < threshold
- False Negative (FN): A ground truth not detected
- True Negative (TN): Does not apply. It would represent a corrected misdetection. In the object detection task there are many possible bounding boxes that should not be detected within an image. Thus, TN would be all possible bounding boxes that were corrrectly not detected (so many possible boxes within an image). That's why it is not used by the metrics.

*threshold*: depending on the metric, it is usually set to 50%, 75% or 95%.

#### Precision, Recall and F1

- Precision is the ability of a model to identify only the relevant objects. It is the percentage of correct positive predictions.
- Recall is the ability of a model to find all the relevant cases (all ground truth bounding boxes). It is the percentage of true positive detected among all relevant ground truths.

Overall: `Percision = Cut the bullshit`, `Recall = FOMO`. Here are their mathematical definitions:

<img src="https://miro.medium.com/max/1068/1*EXa-_699fntpUoRjZeqAFQ.jpeg"/>


#### Micro/Macro/Weighted Average
- Micro-average: aggregates the contributions of all classes to compute the average metric.  Basically, every sample-class pair contributes equally to the overall metric.
- Macro-averaged: compute the metric independently for each class and then take the average, hence all classes equally contribute to the final averaged metric.
- Weighted-averaged: each classes’s contribution to the average is weighted by its sample size.

See [here](https://tomaxent.com/2018/04/27/Micro-and-Macro-average-of-Precision-Recall-and-F-Score/) for an example of their calculation

#### Intersection Over Union (IOU)

Intersection Over Union (IOU) is a measure based on Jaccard Index that evaluates the overlap between two bounding boxes. It requires a ground truth bounding box  and a predicted bounding box . By applying the IOU we can tell if a detection is valid (True Positive) or not (False Positive).

IOU is given by the overlapping area between the predicted bounding box and the ground truth bounding box divided by the area of union between them:  

$$IOU=\frac{area(B_p \cap B_{gt})}{area(B_p \cup B_{gt})}$$

The image below illustrates the IOU between a ground truth bounding box (in blue) and a detected bounding box (in red).

<img src="https://miro.medium.com/max/1400/1*FrmKLxCtkokDC3Yr1wc70w.png" width=600/>
<center>(image source: Jonathan Hui)</center>

####  Mean Average Precision (mAP)

mAP is a fairly complex metric (see [this blog](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173) for a more detailed explanation). 

In short, mAP is calculated by:
    
1. Matching prediction with ground truth object if they overlap above some IoU value. The COCO evaluation protocol introduces one additional step: mAP are averaged across a range of 10 IoU thresholds from 0.5 to 0.95 in increments of 0.05. 
2. Computing the number of TP, FP, and FN separately for each class and IoU threshold. Using these TP, FP, FN to generate a precision-recall curve.
3. Computing the average precision (AP) for each class and IoU threshold. Average Precision (AP) is the Area under the Precision-Recall Curve (AUC of PR-curve).
4. For every class that contains at least one ground truth object, compute the AP by averaging the precision values over all 10 IoU thresholds. Then compute mAP by averaging the per-class AP values over all classes

Refer to [mAP protocol](https://voxel51.com/docs/fiftyone/integrations/coco.html#map-protocol) for the specific mAP calculation steps.


### (2) Running evaluation
We can run evaluation on our samples via [`evaluate_detections( )`](https://voxel51.com/docs/fiftyone/api/fiftyone.core.collections.html?highlight=evaluate_detections#fiftyone.core.collections.SampleCollection.evaluate_detections). Note that this method is available on both the Dataset and DatasetView classes, which means that we can run evaluation on our `predictions_view` to assess the quality of predictions in our dataset.

In [ ]:
# Evaluate the predictions in the model field of our `predictions_view`
# with respect to the objects in the `ground_truth_detections` field

results = predictions_view.evaluate_detections(
    pred_field = "mytest", #the name of the field containing the predicted
    gt_field="ground_truth_detections", #the name of the field containing the ground truth
    eval_key="eval", 
    compute_mAP=True,
    iou=0.5 #the IoU threshold to use to determine matches
)

Evaluating detections...
 100% |███████████████████| 25/25 [459.8ms elapsed, 0s remaining, 54.4 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 25/25 [742.9ms elapsed, 0s remaining, 33.7 samples/s]      


### (3) Aggregate results

The `results` object returned by the evaluation routine provides a number of convenient methods for analyzing our predictions. For example, let’s print a classification report for the top-10 most common classes in the dataset:

In [ ]:
# Get the 10 most common classes in the dataset by setting [:10]
# You could remove [:10] to display all classes

counts = coco_dataset.count_values("ground_truth_detections.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)

              precision    recall  f1-score   support

      person       0.48      0.94      0.64        17
       bench       0.38      1.00      0.56         5
     handbag       0.33      0.25      0.29         4
         car       0.00      0.00      0.00         1
        boat       0.50      0.67      0.57         9
      carrot       0.62      0.56      0.59         9
dining table       0.33      0.50      0.40         4
    suitcase       0.40      0.50      0.44         4
        book       0.12      0.33      0.18         9
     bicycle       0.00      0.00      0.00         0

   micro avg       0.38      0.65      0.48        62
   macro avg       0.32      0.47      0.37        62
weighted avg       0.41      0.65      0.49        62



To visualise the detection performance for specific objects, we can plot their precision-recall (P-R) curves. The P-R curve is generated by sweeping through a range of confidence thresholds on the detections. The area under the P-R curve is averaged over all classes to compute mAP.

In [ ]:
plot = results.plot_pr_curves(classes=["person", "bench", "car"])
plot.show()

### (4) Evaluating model mAP

We can also compute the mean average-precision (mAP) of our detector (all mAP calculations are performed according to the [COCO evaluation protocol](https://cocodataset.org/#detection-eval)):

In [ ]:
print(results.mAP())

0.4497866871508579


### (5) Sample-level analysis
The evaluation routine also populated some new fields on our dataset that contain helpful information that we can use to evaluate our predictions at the sample-level.

In particular, each sample now contains new fields:
- `eval_tp`: the number of true positive (TP) predictions in the sample
- `eval_fp`: the number of false positive (FP) predictions in the sample
- `eval_fn`: the number of false negative (FN) predictions in the sample

In [ ]:
# Our dataset's schema now contains `eval_*` fields
print(coco_dataset)

Name:        2021.10.16.06.56.52
Media type:  image
Num samples: 50
Persistent:  False
Tags:        []
Sample fields:
    id:                      fiftyone.core.fields.ObjectIdField
    filepath:                fiftyone.core.fields.StringField
    tags:                    fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth_detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    ground_truth_coco_id:    fiftyone.core.fields.IntField
    mytest:                  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:                 fiftyone.core.fields.IntField
    eval_fp:                 fiftyone.core.fields.IntField
    eval_fn:                 fiftyone.core.fields.IntField


The individual predicted and ground truth objects also have fields populated on them describing the results of the matching process:
- `eval`: whether the object is a TP/FP/FN
- `eval_id`: the ID of the matching ground truth/predicted object, if any
- `eval_iou`: the IoU between the matching objects, if any

These extra fields were added because we provided the `eval_key` parameter to `evaluate_detections()`. If we had omitted this parameter, then no information would have been recorded on our samples.

In [ ]:
# Our detections have helpful evaluation data on them
sample = predictions_view.first()
print(sample.mytest.detections[0])

<Detection: {
    'id': '616a77ee36fd785cd314eec7',
    'attributes': BaseDict({}),
    'tags': BaseList([]),
    'label': 'elephant',
    'bounding_box': BaseList([
        0.11572663486003876,
        0.34526488184928894,
        0.2370869666337967,
        0.14691317081451416,
    ]),
    'mask': None,
    'confidence': 0.8228263854980469,
    'index': None,
    'eval': 'tp',
    'eval_id': '616a77b536fd785cd314ece9',
    'eval_iou': 0.8585059423309278,
}>


## 4 Exercise
### Exercise 1
Compare and contrast the results of the Faster R-CNN, SSD, and CenterNet. Also try swapping the "backbone" of a network (e.g., ResNet vs. MobileNet) to see if it changes the results. (Hints: change the `MODEL_DATE` and ` MODEL_NAME` in **2(1)-Download the model**)

### Exercise 2
Try changing `min_score_thresh` in **2(4)-Make predictions** to other values (between 0 and 1). How does this affect the results of detections? Why?

Hint: This is where you need to look at
```python
for label, score, box in zip(detections['detection_classes'], detections['detection_scores'], detections['detection_boxes']):
    # Convert to [top-left-x, top-left-y, width, height]            
    ymin, xmin, ymax, xmax = box
    rel_box = [xmin, ymin, (xmax - xmin), (ymax - ymin)]

    if score > min_score_thresh:
        result_detections.append(
            fo.Detection(
                label=classes[label],
                bounding_box=rel_box,
                confidence=score
            )
        )
```

### Exercise 3
Modify some of the input images and see if detection still works. For example, converting the images to grayscale (just uncomment the relevant portions of code in **2(4)-Make predictions**), or splitting your own sub-dataset from the COCO (see **2(3)-Optional**).

*Answer: Open-ended question. This will help you find out where the model does not perform well, or what kind of details it usually captures in the dataset.*

In [24]:
%%html
<marquee style='width: 80%; color: white; font-size:30px;' scrollamount=5 direction="down"
  width="300"
  height="100"
  behavior="alternate"
  style="border:solid"><marquee behavior="alternate" scrollamount=5.5><b>Congratulations, you have reached the ending path of your course!</b></marquee></marquee>

## After this, you will:
- Know about image processing techniques, and image representations
- Know about how deep learning is used in computer vision
- Know that it's called CNN, not some shit like `iMagE Ai`
- Know how Midjourney, DALL-E, etc. works (See week 10)
- Be proud that you had me as your sick ass tutor (jk, would be great to have your feedback. It will be very meaningful to me)
- Able to flex with your friends about all of the above

## For the exam:
- Don't try to aim for 100%, aim to do as best as you can (you can thank me later, this applies to almost every UniMelb exam). Be strategic!
- No coding, theoretical questions will be asked. Some examples:
  - What happens when you change parameter `x` when doing `xxx` method?
  - Why do we need to do `y` when doing `yyy`?
  - Does `aaa` variation of `bbb` method works on `ccc` type of images? Why?
  - What is the correct `ggg` representation of this image?
  - True/False stuffs
- Some mathy questions (very basic maths):
  - Camera geometry
  - CNN parameters/multiplications
  - Object detection (ROI, metrics like percision, recall, etc.)
  - And probably more (but no stuffs like 'proof that derivative of `kkk` is equal to sum squared of `lll`' though)
- Open-ended question (might be a long one - you can probably prepare some templates):
  - What model would be suitable for this?
  - How would you set up your datasets?
  - A method to handle these types of problem?
- In case you need revision, I stored the notebook solutions here: https://github.com/tuankhoin/COMP90086-Practical-Solutions . Give it a star ⭐ to make me look cool on GitHub 👌!

For now, get on the beers 🍺. After the exams, get on another beer 🍺! Good luck everyone! May your next journeys have more joy and less (UniMelb) pain.